# Step 1 is to setup the Google collab environment

In [ ]:
!pip install -U featuretools

In [ ]:
!pip install nvcc4jupyter
#!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

In [ ]:
%%writefile main.cu
#include <stdio.h>
#include <iostream>

#define N 10

__global__ void add(int* a, int* b, int* c){
    int index = threadIdx.x; // The CUDA runtime contexts defines this for us.
    if (index < N) {
      a[index] = b[index] + c[index];
    }
}

void print(int* nums, int size) {

    std::cout << std::endl << '[';

    for (int i = 0; i < size; i++) {
      if (i < (size - 1)) {
        std::cout << nums[i] << ", ";
      }
      else {
        std::cout << nums[i];
      }
    }

    std::cout << std::endl << ']';

}

int main() {

    int* a_device, *b_device, *c_device;
    int* a_host, *b_host, *c_host;

    // Allocate space on the host for A, B, and C.
    a_host = (int*) malloc(sizeof(int) * N);
    b_host = (int*) malloc(sizeof(int) * N);
    c_host = (int*) malloc(sizeof(int) * N);


    // Write data to B and C.
    for (int i = 0; i < N; i++) {
      b_host[i] = 10;
      c_host[i] = 10;
    }

    // Allocate space on the GPU for A, B, and C.
    cudaMalloc((void**)&a_device, N * sizeof(int));
    cudaMalloc((void**)&b_device, N * sizeof(int));
    cudaMalloc((void**)&c_device, N * sizeof(int));

    // Copy A and B to the device.
    cudaMemcpy(a_device, a_host, N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(b_device, b_host, N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(c_device, c_host, N * sizeof(int), cudaMemcpyHostToDevice);

    // Call add.
    int num_blocks = 256;
    int num_threads_per_block = 1;
    add<<<num_blocks, num_threads_per_block>>>(a_device, b_device, c_device);

    // Copy A back to the host.
    cudaMemcpy(a_host, a_device, N * sizeof(int), cudaMemcpyDeviceToHost);

    print(a_host, N);


    return 0;
}

In [ ]:
%%script bash
nvcc main.cu -o a.out

In [ ]:
%%script bash
./a.out